# **Setup and Load Data**

# Install dependecies and setup
Tensorflow for pipelining, opencv for cleaning dataset, matplotlib for visualization

In [28]:

%pip install tensorflow tensorflow-gpu opencv-python matplotlib 

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'd:\project\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Validation

In [29]:
%pip list

Package                      Version
---------------------------- -----------
absl-py                      1.2.0
argon2-cffi                  21.3.0
argon2-cffi-bindings         21.2.0
asttokens                    2.0.8
astunparse                   1.6.3
attrs                        22.1.0
backcall                     0.2.0
beautifulsoup4               4.11.1
bleach                       5.0.1
cachetools                   5.2.0
certifi                      2022.6.15.1
cffi                         1.15.1
charset-normalizer           2.1.1
colorama                     0.4.5
cycler                       0.11.0
debugpy                      1.6.3
decorator                    5.1.1
defusedxml                   0.7.1
entrypoints                  0.4
executing                    1.0.0
fastjsonschema               2.16.1
flatbuffers                  2.0.7
fonttools                    4.37.1
gast                         0.4.0
google-auth                  2.11.0
google-auth-oauthlib         0.4.6

You should consider upgrading via the 'd:\project\venv\Scripts\python.exe -m pip install --upgrade pip' command.


Importing dependecies

In [30]:
import tensorflow as tf
import os 
#navigate through file structures
from tensorflow import keras
from keras import layers

Limiting gpu vrm usage 

In [31]:
import cv2
import imghdr
from matplotlib import pyplot as plt

In [37]:
os.chdir('D:\\project')
data_dir = 'trainData'

In [39]:
os.listdir(data_dir)

['banana', 'mold', 'over_ripe', 'ripe', 'rotten', 'unripe']

Pre-processing

In [34]:
import numpy as np
from matplotlib import pyplot as plt

In [40]:
data = tf.keras.utils.image_dataset_from_directory('trainData')

Found 5264 files belonging to 6 classes.


In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()

In [ ]:
# Images represented as numpy arrays
batch[0].shape

In [ ]:
batch[1]